In [2]:
import os
import json
import pickle
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook
from catboost import CatBoostRegressor

# Displaying pd Dataframe options
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', 1000)

/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [3]:
%%time
# Get datas from pickle
train_df = pd.read_pickle('train_df.pickle')
test_df = pd.read_pickle('test_df.pickle')
test_df['transactionRevenue'] = np.nan

full_df = pd.concat([train_df, test_df], axis=0)

full_df['date'] = pd.to_datetime(full_df['date'], format='%Y%m%d')
full_df['weekday'] = full_df['date'].dt.weekday_name
full_df['day_off'] = (full_df['date'].dt.dayofweek > 4).astype(int)



CPU times: user 7.41 s, sys: 4.42 s, total: 11.8 s
Wall time: 13.1 s


In [4]:
count_user_by_visits = {}
def check_count_by_visits(Id):
    if Id in count_user_by_visits:
        count_user_by_visits[Id]+= 1
    else:
        count_user_by_visits[Id]= 1
    
    return count_user_by_visits[Id]
full_df['visit_index'] = [check_count_by_visits(user) for user in full_df['fullVisitorId']]

In [ ]:
cat_features = ['channelGrouping','isMobile','browser','deviceCategory','operatingSystem',
               'country','region','subContinent','networkDomain','continent','newVisits',
               'medium', 'isTrueDirect', 'source', 'isVideoAd', 'slot','page', 'day_off', 
                'weekday']
num_features = ['visitNumber','pageviews','hits', 'visit_index']

categorical_features=[i for i, x in enumerate(X_train.columns) if x in cat_features]

In [ ]:
full_df[num_features] = test_df[num_features].replace('no_key',0).fillna(0)
full_df[cat_features] = test_df[cat_features].fillna('other')

In [ ]:
X_train, X_test = full_df[full_df['transactionRevenue'] >= 0], full_df[~(full_df['transactionRevenue'] >= 0)]
y_train = X_train['transactionRevenue']

X_train, X_test = X_train[cat_features+num_features], X_test[['fullVisitorId']+cat_features+num_features]

In [ ]:
# 13) Интегрировать код в блок с обучением катбуста. Для снижения снижения лернрейта

In [ ]:
model_Cat = CatBoostRegressor(iterations=100, thread_count=4, learning_rate=0.4,
                             depth=10, border_count=100, has_time=True,
                             counter_calc_method='SkipTest')
model_Cat.fit(X_train, y_train, cat_features=categorical_features)

In [ ]:
X_test['prediction'] = model_Cat.predict(X_test.iloc[:,1:])
X_test['prediction'] = X_test['prediction']*(X_test['prediction'] >= 0)
X_test['prediction'] = np.exp(X_test['prediction']) - 1
pred_dict = dict(X_test.groupby('fullVisitorId')['prediction'].sum())

In [ ]:
subm = pd.read_csv('sample_submission.csv')
subm['PredictedLogRevenue'] = subm['fullVisitorId'].map(pred_dict).fillna(0)
subm['PredictedLogRevenue'] = np.log(subm['PredictedLogRevenue']+1)
subm.to_csv('fifth_subm.csv', sep=',', index=None)